In [ ]:
!gcloud services enable ml.googleapis.com
!gcloud services enable compute.googleapis.com

### Create the bucket 

Create it from User interface or from here by running cells below (if not exists)

In [ ]:
BUCKET_NAME = 'ribtdap-ds-aiplatform'
VERSION = 'v1'

In [ ]:
!gsutil mb gs://$BUCKET_NAME/

### Create temporary local folder

In [ ]:
!mkdir $VERSION

### Create model 

In [ ]:
import argparse
import pickle
import pandas as pd
from sklearn.linear_model import LinearRegression as LR

from google.cloud import bigquery
from google.cloud import storage

# bigquery
client = bigquery.Client()

# Perform a query.
QUERY = (
    """
    SELECT passenger_count, trip_distance , fare_amount as label
    FROM `nyc-tlc.yellow.trips`
    WHERE trip_distance > 0 AND fare_amount > 0
    ORDER BY rand()
    LIMIT 1000
    """
    )
query_job = client.query(QUERY)  # API request
df = query_job.result().to_dataframe()

# save file locally
df.to_csv(VERSION+'/data.csv')

# preprocess
df = df.fillna(0)

# split
TRAINING_SPLIT = 0.8
df_train = df.loc[:int(TRAINING_SPLIT * len(df)), :]
df_test = df.loc[int(TRAINING_SPLIT * len(df)):, :]

# save files locally
df_train.to_csv(VERSION+'/data_train.csv')
df_test.to_csv(VERSION+'/data_test.csv')

# train model
lr = LR()
X = df_train.drop(['label'], axis=1)
Y = df_train['label']
lr.fit(X, Y)

# save model
with open(VERSION+'/model.pkl', 'wb') as model_file:
    pickle.dump(lr, model_file)


# if we want to save a single file in gcs
#gcs = storage.Client()
#gcs.get_bucket(BUCKET_NAME).blob('model.pkl').upload_from_filename('model.pkl')

In [ ]:
# save notebook in local folder
!cp poc_cloudrun.ipynb $VERSION

# Copy all the local folder in GCS
!gsutil cp -r $VERSION gs://$BUCKET_NAME

### Verify files in Cloud Storage

In [ ]:
!gsutil ls gs://$BUCKET_NAME/$VERSION/

### Serve the model

In [ ]:
MODEL_NAME = "model_aiplatform"
MODEL_VERSION = MODEL_NAME + "_v1"

In [ ]:
!gcloud ai-platform models create $MODEL_NAME --regions europe-west1

Create a version that points to your model file in Cloud Storage:

In [ ]:
!gcloud ai-platform versions create $MODEL_VERSION \
  --model=$MODEL_NAME \
  --framework=scikit-learn \
  --origin=gs://$BUCKET_NAME/$VERSION/ \
  --python-version=3.7 \
  --runtime-version=2.1

### Make predictions

Check README file.

### Clean up

To delete all resources you created in this tutorial, run the following commands:

In [ ]:
# Delete local folder
!rm -r $VERSION

In [ ]:
# Delete the model version
!gcloud ai-platform versions delete $MODEL_VERSION --model=$MODEL_NAME --quiet

# Delete the model
!gcloud ai-platform models delete $MODEL_NAME --quiet